In [ ]:
#| default_exp llama

##### this nb contains custom llama index classes asa module, as well as snippets to work with llama

In [ ]:
!nbdev_export

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'..')

In [ ]:
#| export
from pathlib import Path
from typing import Any, Dict, List, Optional

import pandas as pd

from llama_index.readers.base import BaseReader
from llama_index.schema import Document
from llama_index import VectorStoreIndex
from llama_index.response_synthesizers.base import BaseSynthesizer
from llama_index.prompts import BasePromptTemplate
from llama_index.indices.service_context import ServiceContext
from typing import Any, List, Optional, Sequence
from llama_index.types import RESPONSE_TEXT_TYPE

from llama_index.schema import Node, NodeWithScore
from llama_index.response_synthesizers import get_response_synthesizer
from llama_index.response_synthesizers import ResponseMode, get_response_synthesizer,Accumulate

from llama_index.response_synthesizers.base import BaseSynthesizer
from llama_index.prompts.base import BasePromptTemplate
from llama_index.types import RESPONSE_TEXT_TYPE

import asyncio
from typing import Any, List, Optional, Sequence

from llama_index.async_utils import run_async_tasks
from llama_index.indices.service_context import ServiceContext
from llama_index.prompts import BasePromptTemplate
from llama_index.prompts.default_prompts import DEFAULT_TEXT_QA_PROMPT
from llama_index.response_synthesizers.base import BaseSynthesizer
from llama_index.types import RESPONSE_TEXT_TYPE

from llama_index.prompts.base import PromptTemplate

import os
import asyncio
from datetime import datetime
from typing import Any, List, Optional, Sequence

## funcs

In [ ]:
#| export
def get_embedding(q):
    return torch.Tensor(service_context.embed_model._get_query_embedding(q))
    

In [ ]:
#| export
def cos(emb1,emb2):
    return F.cosine_similarity(torch.Tensor(emb1), torch.Tensor(emb2), dim=0)

## custom llama index classes

#### pandas to docs

In [ ]:
#| export
class PandasReader(BaseReader):


    def __init__(
        self,
        *args: Any,
        concat_rows: bool = True,
        col_joiner: str = ", ",
        row_joiner: str = "\n",
        pandas_config: dict = {},
        **kwargs: Any
    ) -> None:
        """Init params."""
        super().__init__(*args, **kwargs)
        self._concat_rows = concat_rows
        self._col_joiner = col_joiner
        self._row_joiner = row_joiner
        self._pandas_config = pandas_config

    def load_data(
        self, df, extra_info: Optional[Dict] = None
    ) -> List[Document]:
        """Parse file."""
        text_list = df.apply(
            lambda row: (self._col_joiner).join(row.astype(str).tolist()), axis=1
        ).tolist()

        if self._concat_rows:
            return [
                Document(
                    text=(self._row_joiner).join(text_list), metadata=extra_info or {}
                )
            ]
        else:
            return [
                Document(text=text, metadata=extra_info or {}) for text in text_list
            ]

### custom prompt

In [ ]:
#| export
prompt =  """In order to {query_str}, please extract information from 'the input', please follow this schema:

For every organization mentioned, collect its name, details, location, and any URLs associated with it.
For every person mentioned, capture their name, position, education, location, activity, LinkedIn profile, and other related URLs.
For every post or activity related to a person, collect its details and related URLs.
If the location of an organization or person is mentioned, note it down.
For every relationship, identify the type and the entities involved. This can be a person working at an organization, someone commenting on a post, or other types of relations.
Use the structure provided to organize the data in a logical manner.

GRAPH_SCHEMA:
```{{
    "Organization": [ ["C001", "", "", []] ], // [ID, Name, Details, URLs]
    "Person": [ ["P001", "", "", []] ], // [ID, Name, Position, URLs]
    "Post": [ ["PP001", "", []] ],   // [ID, Details, URLs]
    "Location": [ ["L001", ""] ],    // [ID, Name]
    "Education": [ ["E001", ""] ],   // [ID, Name]
    "Activity": [ ["A001", "", []] ],  // [ID, Details, URLs]
    "Relationship": [ ["R001", "RELATIONSHIP_TYPE", "P001", "C001", "", []] ] // [ID, Type, Start, End, Details, URLs]
}}

parse 'the input' below:
{context_str}

Return graph

"""

In [ ]:
#| export
from llama_index.prompts.base import PromptTemplate
class CustomPrompt(PromptTemplate):
    def __init__(self, template: str, **kwargs: Any):
        super().__init__(template=template, **kwargs)

In [ ]:
prompt = CustomPrompt('summarize')

### synthesizers 
##### context Nodes  -> LLM output

In [ ]:
#| export

#prompt specified as CustomPrompt
class Accumulate2Graph(BaseSynthesizer):
    
    """Accumulate responses from multiple text chunks and save them into files."""

    def __init__(
        self,
        text_qa_template: Optional[BasePromptTemplate] = None,
        service_context: Optional[ServiceContext] = None,
        save_dir: str = 'responses',
        streaming: bool = False,
        use_async: bool = False,
        prompt:str = ''
    ) -> None:
        super().__init__(service_context=service_context, streaming=streaming)
        self._text_qa_template = text_qa_template or DEFAULT_TEXT_QA_PROMPT
        self._use_async = use_async

        self.save_dir = save_dir
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        self.custom_template = CustomPrompt(prompt)#prompt goes here

    async def aget_response(
        self,
        query_str: str,
        text_chunks: Sequence[str],
        separator: str = "\n---------------------\n",
        **response_kwargs: Any,
    ) -> RESPONSE_TEXT_TYPE:
        if self._streaming:
            raise ValueError("Unable to stream in Accumulate response mode")

        async def process_chunk(query_str, text_chunk):
            output = await self._give_responses(query_str, text_chunk, use_async=True)
            save_response_to_file(query_str, output[0],self.save_dir)  # Assuming each chunk returns a list with a single item.
            return output

        tasks = [process_chunk(query_str, text_chunk) for text_chunk in text_chunks]
        await asyncio.gather(*tasks)

        return f"Responses for query {query_str} have been saved to the directory: {self.save_dir}"

    def get_response(
        self,
        query_str: str,
        text_chunks: Sequence[str],
        separator: str = "\n---------------------\n",
        **response_kwargs: Any,
    ) -> RESPONSE_TEXT_TYPE:
        if self._streaming:
            raise ValueError("Unable to stream in Accumulate response mode")

        for text_chunk in text_chunks:
            output = self._give_responses(query_str, text_chunk)
            if output:
                save_response_to_file(query_str, output[0],self.save_dir)
            if not output:
                print(f"No response for text_chunk: {text_chunk}")
            else:
                save_response_to_file(query_str, output[0],self.save_dir)
                

        return f"Responses for query {query_str} have been saved to the directory: {self.save_dir}"

    def _give_responses(
        self, query_str: str, text_chunk: str, use_async: bool = False
    ) -> List[Any]:
        text_qa_template = self.custom_template.partial_format(query_str=query_str)
        text_chunks = self._service_context.prompt_helper.repack(text_qa_template, [text_chunk])
        predictor = self._service_context.llm_predictor.apredict if use_async else self._service_context.llm_predictor.predict
        

        return [predictor(text_qa_template, context_str=cur_text_chunk) for cur_text_chunk in text_chunks]

response modes
[responce modes](https://gpt-index.readthedocs.io/en/stable/core_modules/query_modules/query_engine/response_modes.html)

In [ ]:
prompt

CustomPrompt(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=[], kwargs={}, output_parser=None, template='summarize')

In [ ]:
service_context = ServiceContext.from_defaults()
synthesizer = Accumulate2Graph(service_context=service_context,prompt=prompt)

synthesizer.synthesize('parse contact details of a person',
    nodes = [NodeWithScore(node=Node(text=c), score=1.0) for c in chunks])

TypeError: expected str, got CustomPrompt

## snippets

### openai

In [ ]:
#| export
import openai
from mytools.tools import get_api_key
from mytools.paths import *
openai.api_key = get_api_key(api_path, 'openai')

### documents

In [ ]:
#from folder
documents = SimpleDirectoryReader('data').load_data()

In [ ]:
#from pandas 
df = pd.DataFrame()
pdreader = PandasReader()
documents = pdreader.load_data(df)

### index

In [ ]:

from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

### query

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)

None


### chat

#### memory

In [ ]:
#simple
from llama_index.memory import ChatMemoryBuffer
memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

#### chat

In [ ]:
chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt="You are a chatbot, able to have normal interactions, as well as talk about an essay discussing Paul Grahams life.",
)

response = chat_engine.chat("Hello!")

In [ ]:
response

AgentChatResponse(response='Hello! How can I assist you today?', sources=[ToolOutput(content='system: Context information is below.\n--------------------\n\n--------------------\nYou are a chatbot, able to have normal interactions, as well as talk about an essay discussing Paul Grahams life.', tool_name='retriever', raw_input={'message': 'Hello!'}, raw_output=ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='Context information is below.\n--------------------\n\n--------------------\nYou are a chatbot, able to have normal interactions, as well as talk about an essay discussing Paul Grahams life.', additional_kwargs={}))], source_nodes=[])

In [ ]:
chat_engine.reset()

### specify LLM

In [ ]:
from llama_index.llms import OpenAI
llm = OpenAI(temperature=0.1, model="gpt-4")
service_context = ServiceContext.from_defaults(llm=llm)

query_engine = index.as_query_engine()
response = query_engine.query("What did the author do after his time at Y Combinator?")
